# 라이브러리 임포트

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
!pip install finance-datareader
import FinanceDataReader as fdr
from statsmodels.tsa.seasonal import seasonal_decompose

from lightgbm import LGBMRegressor

import os 
from tqdm import tqdm
import warnings 
warnings.filterwarnings(action = 'ignore') 
pd.options.display.max_columns = None

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 데이터셋 로드

In [3]:
path = '/content/drive/MyDrive/stock_2/'

stock_list = pd.read_csv(path + 'stock_list.csv')
stock_list['종목코드'] = stock_list['종목코드'].apply(lambda x: str(x).zfill(6))
stock_codes = np.sort(stock_list['종목코드'].values)

In [4]:
# start_date = '20210104'
# end_date = '20211105'

# start_weekday = pd.to_datetime(start_date).weekday()
# max_weeknum = pd.to_datetime(end_date).strftime('%V')
# Business_days = pd.DataFrame(pd.date_range(start_date,end_date,freq='B'), columns = ['Date'])

# print(f'WEEKDAY of "start_date" : {start_weekday}')
# print(f'NUM of WEEKS to "end_date" : {max_weeknum}')
# print(f'HOW MANY "Business_days" : {Business_days.shape}', )
# display(Business_days.head())

# 전처리를 위한 함수 정의

In [5]:
# 이동평균 추가하는 함수 정의 
def add_rolling_mean(df:pd.DataFrame, col_name:str, window_list:list)->pd.DataFrame:
  for window in window_list:
    df[col_name + f'_rolling{window}'] = df[col_name].rolling(window).mean().fillna(method = 'bfill') 
  return df

# 컬럼값을 lag하는 함수 정의
def lag_features(df:pd.DataFrame, col_list:list, lag_num:int)->pd.DataFrame:
  for col in col_list:
    df[col] = df[col].shift(lag_num).fillna(method = 'ffill').fillna(method = 'bfill')
  return df

# 종목별 전처리

In [6]:
stock_df_dict = {}

# 종목별로 stock_df 만들어서 딕셔너리 stock_df_dict에 저장하기 
for stock_code in tqdm(stock_codes): 

  # 날짜 설정
  dates = pd.date_range('20110104', '20211203', freq = 'B')
  df = pd.DataFrame(dates, columns = ['Date'])

  # 연, 월, 주, 일, 요일, 월별주수 정보 추가
  df['Year'] = df['Date'].dt.year
  df['Month'] = df['Date'].dt.month
  df['Week'] = df['Date'].dt.week
  df['Day'] = df['Date'].dt.day
  df['Weekday'] = df['Date'].dt.weekday
  df['WeekNum'] = np.ceil(df['Day']/7).astype(int)

  # 20110101 ~ 20210910 까지의 주가 정보 가져오기
  stock_value = fdr.DataReader(stock_code, start = '20110104', end = '20211128').reset_index()
  df = pd.merge(df, stock_value, on = 'Date', how = 'left')
  df[stock_value.columns] = df[stock_value.columns].ffill() # 결측치를 이전 값으로 대체 (공휴일 등 쉬는 날에 대하여)
  df = df.dropna() # 남아있는 결측치는 상장전이므로 drop 

  # 종가 데이터 시계열 분해
  df = df.set_index('Date') 
  decomposition = seasonal_decompose(df[:-5]['Close'], model='multiplicative') 
  df['Close_trend'] = decomposition.trend.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 추세 
  df['Close_seosonal'] = decomposition.seasonal.fillna(method = 'ffill').fillna(method = 'bfill') # 종가 계절성 

  # 5일, 20일, 60일, 120일, 240일 이동평균 추가 
  df = add_rolling_mean(df, 'Close', [5, 20, 60, 120, 240]) 

  # lag features 
  col_list = list(df.iloc[:, 6:].columns)
  col_list.remove('Close')
  df = lag_features(df, col_list, 5)
  df.drop(df.index[:5])

  df = df.reset_index()
  df['Index'] = df.index 
  stock_df_dict[stock_code] = df

100%|██████████| 370/370 [02:50<00:00,  2.17it/s]


# 모델링 (XGBoost)

In [7]:
# SEED 설정 
SEED = 42

# NMAE 함수 정의
def NMAE(true, pred):
 return np.mean(abs(true - pred) / true) * 100 

## public

In [8]:
# public 
model_oof_score = []
model_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (public)
  train = stock_df_dict[stock_code][:-25]
  test = stock_df_dict[stock_code][-25:-20]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  model = LGBMRegressor(random_state = SEED)
  model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100, eval_metric = 'mae', verbose = 100)

  model_oof = model.predict(X_valid)
  model_pred = model.predict(X_test)
  model_pred_dict[stock_code] = model_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, model_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Public OOF NMAE Score : {nmae:.6f}\n")
  model_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Public OOF NMAE Score : {np.mean(model_oof_score)}')

  0%|          | 1/370 [00:00<00:53,  6.90it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 504.875	valid_0's l2: 591566
Did not meet early stopping. Best iteration is:
[64]	valid_0's l1: 418.854	valid_0's l2: 435299

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.485637


[종목코드 000080 종목명 하이트진로 학습 및 예측]
Training until validation scores don't improve for 100 rounds.


  1%|          | 3/370 [00:00<00:54,  6.79it/s]

[100]	valid_0's l1: 740.368	valid_0's l2: 882995
Did not meet early stopping. Best iteration is:
[91]	valid_0's l1: 719.659	valid_0's l2: 875713

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 2.006294


[종목코드 000100 종목명 유한양행 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 696.559	valid_0's l2: 663989
Did not meet early stopping. Best iteration is:
[55]	valid_0's l1: 472.634	valid_0's l2: 400773

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 0.777990


[종목코드 000120 종목명 CJ대한통운 학습 및 예측]
Training until validation scores don't improve for 100 rounds.


  1%|▏         | 5/370 [00:00<00:52,  6.99it/s]

...

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 2375.02	valid_0's l2: 8.31456e+06
Did not meet early stopping. Best iteration is:
[78]	valid_0's l1: 2249.15	valid_0's l2: 7.58717e+06

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 4.178533


[종목코드 336370 종목명 솔루스첨단소재 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 978.246	valid_0's l2: 1.60571e+06
Did not meet early stopping. Best iteration is:
[80]	valid_0's l1: 837.963	valid_0's l2: 1.04483e+06

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.022934


[종목코드 347860 종목명 알체라 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 3827.21	valid_0's l2: 1.7599e+07
Did not meet early stopping. Best iteration is:
[5]	valid_0's l1: 3131.76	valid_0's l2: 1.12459e+07

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 8.364764


[종목코드 348150 종목명 고바이오랩 학습 및 예측]
Training until validation scores don't improve for 100

 99%|█████████▉| 367/370 [00:49<00:00, 18.22it/s]


2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 3.600405


[종목코드 348210 종목명 넥스틴 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1622.97	valid_0's l2: 3.64663e+06
Did not meet early stopping. Best iteration is:
[16]	valid_0's l1: 1001.09	valid_0's l2: 1.60316e+06

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.897861


[종목코드 352820 종목명 하이브 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 33428.9	valid_0's l2: 1.28407e+09
Did not meet early stopping. Best iteration is:
[64]	valid_0's l1: 33068.4	valid_0's l2: 1.24309e+09

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 9.749024


[종목코드 357780 종목명 솔브레인 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 5416.16	valid_0's l2: 3.54908e+07
Did not meet early stopping. Best iteration is:
[21]	valid_0's l1: 3522.1	valid_0's l2: 2.65402e+07

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 1.3

100%|██████████| 370/370 [00:49<00:00,  7.45it/s]

[100]	valid_0's l1: 942.372	valid_0's l2: 1.04535e+06
Did not meet early stopping. Best iteration is:
[97]	valid_0's l1: 914.501	valid_0's l2: 991571

2021-10-25 ~ 2021-10-29 기간동안의 Public OOF NMAE Score : 5.238448

전체 종목에 대한 평균 Public OOF NMAE Score : 2.813723527955485


In [9]:
model_public_sub = pd.DataFrame({'Day' : pd.date_range('20211101', '20211105', freq = 'B')})

for stock_code, pred in model_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  model_public_sub = pd.concat([model_public_sub, stock_pred], axis = 1)

model_public_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29889.657078,37788.947299,61587.095578,147803.76888,103845.308066,16875.345887,47152.899014,85094.421061,99148.736520,725721.243542,50555.726542,248348.687793,33855.491438,54439.590736,16763.206868,2405.908078,28396.932081,5304.012955,336471.904023,51888.744001,13258.674533,32377.962076,9794.846346,7671.029963,30769.611923,142164.772353,46380.018871,85025.857741,46228.695364,251719.658228,288527.937752,83227.805793,105378.435112,34064.574384,28992.870467,33934.452441,156979.864601,210124.133035,97844.607999,98809.412408,302795.121238,65305.587288,33702.768963,70107.938206,63818.287159,13267.564213,65409.234518,272254.448613,41902.065036,735235.249091,190665.195459,9193.381633,8910.205407,32891.550562,524605.246067,20148.505833,4753.845892,88672.536632,58309.866484,157351.868913,105901.441482,103613.916739,45039.243164,31089.379789,137166.820721,60288.961378,540875.198955,68524.265666,45734.493080,104869.431468,27785.566079,205726.672182,239357.555729,29383.582609,87053.678675,170312.818294,269569.120239,45552.763357,87117.497892,84371.570095,5387.657752,80408.065960,334660.869844,6201.144581,23587.514469,49016.908438,13290.264844,312265.007901,48175.769354,155788.732438,14789.446216,58170.077851,104477.981912,81171.414225,6756.495753,107397.146841,11455.178547,87748.689196,12418.208428,33218.295605,4561.076568,25271.176049,120891.352143,42788.761768,6309.820088,35169.81

## private

In [10]:
# private
lr_oof_score = []
lr_pred_dict = {}

for stock_code, df in tqdm(stock_df_dict.items()):
  stock_name = str(stock_list.loc[stock_list['종목코드'] == stock_code]['종목명'].values).strip("['']")
  print(f"\n[종목코드 {stock_code} 종목명 {stock_name} 학습 및 예측]")

  # split train, valid, test (private)
  train = stock_df_dict[stock_code][:-5]
  test = stock_df_dict[stock_code][-5:]

  X_train = train[:-5].drop(columns = ['Date', 'Close'])
  y_train = train[:-5]['Close']
  X_valid = train[-5:].drop(columns = ['Date', 'Close'])
  y_valid = train[-5:]['Close']
  X_test = test.drop(columns = ['Date', 'Close'])  

  # xgboost 
  model = LGBMRegressor(random_state = SEED, metric='mae')
  model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 100, eval_metric = 'mae', verbose = 100)

  model_oof = model.predict(X_valid)
  model_pred = model.predict(X_test)
  model_pred_dict[stock_code] = model_pred

  # validation NMAE score 
  nmae = NMAE(y_valid, model_oof)
  print(f"\n{train[-5:]['Date'].min().strftime('%Y-%m-%d')} ~ {train[-5:]['Date'].max().strftime('%Y-%m-%d')} 기간동안의 Public OOF NMAE Score : {nmae:.6f}\n")
  model_oof_score.append(nmae)
  print('=' * 120)

print(f'전체 종목에 대한 평균 Private OOF NMAE Score : {np.mean(model_oof_score)}')

  0%|          | 1/370 [00:00<00:53,  6.91it/s]


[종목코드 000060 종목명 메리츠화재 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 698.8
Did not meet early stopping. Best iteration is:
[71]	valid_0's l1: 655.284

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 2.212311


[종목코드 000080 종목명 하이트진로 학습 및 예측]
Training until validation scores don't improve for 100 rounds.


  1%|          | 3/370 [00:00<00:52,  6.98it/s]

[100]	valid_0's l1: 452.181
Did not meet early stopping. Best iteration is:
[23]	valid_0's l1: 104.413

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 0.316197


[종목코드 000100 종목명 유한양행 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 820.961
Did not meet early stopping. Best iteration is:
[42]	valid_0's l1: 343.749

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 0.575596


[종목코드 000120 종목명 CJ대한통운 학습 및 예측]
Training until validation scores don't improve for 100 rounds.


  1%|▏         | 5/370 [00:00<00:51,  7.06it/s]

...

[100]	valid_0's l1: 11.1229
Did not meet early stopping. Best iteration is:
[40]	valid_0's l1: 5.41026

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 0.095552


[종목코드 330860 종목명 네패스아크 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 2575.01
Did not meet early stopping. Best iteration is:
[19]	valid_0's l1: 1986.32

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 4.119554


[종목코드 336260 종목명 두산퓨얼셀 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 745.738
Did not meet early stopping. Best iteration is:
[24]	valid_0's l1: 586.627

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 1.150737


[종목코드 336370 종목명 솔루스첨단소재 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1421.43
Did not meet early stopping. Best iteration is:
[87]	valid_0's l1: 1106.13

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 1.143717


[종목코드 347860 종목명 알체라 학습 및 예측]


 99%|█████████▉| 367/370 [00:49<00:00, 17.58it/s]

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 4502.9
Did not meet early stopping. Best iteration is:
[5]	valid_0's l1: 485.381

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 1.327027


[종목코드 348150 종목명 고바이오랩 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 3333.53
Did not meet early stopping. Best iteration is:
[61]	valid_0's l1: 3246.3

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 15.231468


[종목코드 348210 종목명 넥스틴 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1032.26
Did not meet early stopping. Best iteration is:
[11]	valid_0's l1: 296.129

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 0.548259


[종목코드 352820 종목명 하이브 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 8233.62
Did not meet early stopping. Best iteration is:
[63]	valid_0's l1: 78.6302

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE S

100%|██████████| 370/370 [00:49<00:00,  7.44it/s]

[100]	valid_0's l1: 754.273
Did not meet early stopping. Best iteration is:
[59]	valid_0's l1: 692.301

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 2.776471


[종목코드 950130 종목명 엑세스바이오 학습 및 예측]
Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 1308.89
Did not meet early stopping. Best iteration is:
[16]	valid_0's l1: 281.311

2021-11-22 ~ 2021-11-26 기간동안의 Public OOF NMAE Score : 1.793415

전체 종목에 대한 평균 Private OOF NMAE Score : 2.611929523421333


In [11]:
model_private_sub = pd.DataFrame({'Day' : pd.date_range('20211129', '20211203', freq = 'B')})

for stock_code, pred in model_pred_dict.items():
  stock_pred = pd.DataFrame({stock_code : pred})
  model_private_sub = pd.concat([model_private_sub, stock_pred], axis = 1)

model_private_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-29,29296.468591,32277.165778,60539.548860,136721.414716,125739.336049,16725.355514,45997.394116,82526.760451,109452.358423,679490.881507,47556.290786,218068.103729,33061.256763,62682.166713,16400.133982,1914.625774,25647.877927,5122.995633,316638.22918,45884.130121,13018.352044,30814.503811,10645.437454,7759.396576,28703.281618,138779.386228,41295.966513,79633.965935,41497.399774,237320.129402,289022.532101,82728.910543,97060.903835,32266.081550,25277.501317,33637.308253,138220.175674,208698.348359,99105.714082,100439.343783,286652.101342,56308.80707,32983.519353,74707.879735,65503.140290,12852.322137,54700.451650,228065.344098,40375.403340,728650.177284,175042.459917,9089.446420,9221.909383,32652.497690,469747.633067,17721.730443,5037.599212,78007.777318,54329.48692,170717.246216,91029.943545,99292.192677,36238.657770,28604.689162,118001.276933,58019.406982,504220.299785,74082.869216,42928.701427,89854.545982,24885.119538,250516.977718,215537.369652,26744.480772,78131.231193,155417.359699,244456.216391,44245.559635,79206.079369,81722.522958,5087.871324,84513.936650,318584.817496,6199.298324,23607.101705,47123.138616,11171.358378,310827.337678,45265.840564,155595.554072,13586.44524,45613.496016,111067.635299,74402.258034,6480.519908,100682.772869,11147.869984,95890.883670,10016.939036,39428.743136,4347.444375,22847.076160,116229.917772,39042.344153,5570.112088,34099.213147

# Submission

In [12]:
model_sub = pd.concat([model_public_sub, model_private_sub], ignore_index = True)
model_sub

,Day,000060,000080,000100,000120,000150,000240,000250,000270,000660,000670,000720,000810,000880,000990,001230,001440,001450,001740,002380,002790,003000,003090,003380,003410,003490,003670,003800,004000,004020,004170,004370,004490,004800,004990,005250,005290,005300,005380,005385,005387,005490,005830,005850,005930,005935,005940,006260,006280,006360,006400,006650,006730,006800,007070,007310,007390,008560,008770,008930,009150,009240,009540,009830,009900,010060,010120,010130,010620,010780,010950,011000,011070,011170,011200,011210,011780,012330,012450,012510,012750,013120,013890,014680,015750,015760,016360,016380,017670,017800,018260,018880,019170,020150,021240,022100,023530,024110,025900,025980,026960,027360,028050,028260,028300,028670,029780,030000,030190,030200,030520,031390,031430,032190,032300,032350,032500,032620,032640,032830,033290,033310,033640,033780,034020,034220,034230,034730,035080,035250,035420,035600,035720,035760,035890,035900,036030,036460,036490,036540,036570,036810,036830,036930,038500,039030,039200,039490,039840,041190,041510,041960,042000,042660,042700,043150,046890,047040,047050,047810,048260,048410,048530,049070,051600,051900,051905,051910,051915,052400,052690,053030,053800,055550,056190,058470,058820,060150,060250,060720,061970,064260,064350,064550,064760,065660,066570,066575,066970,067000,067160,067310,067630,068240,068270,068760,069080,069500,069620,069960,071050,073240,074600,078020,078130,078340,078600,078930,081660,082270,083790,084110,084370,084850,084990,085660,086280,086390,086450,086790,086900,088350,088800,089030,089970,089980,090430,090460,091700,091990,092040,092190,095610,095660,095700,096530,096770,097950,098460,099320,100090,101490,102110,102710,102780,104480,105560,108320,110790,111770,112040,112610,120110,121800,122450,122630,122870,128940,131290,131370,131970,137400,138040,138930,139130,139480,140410,140860,141080,143240,144510,145020,151860,151910,153130,161390,161890,166090,175330,178320,178920,180640,181710,183300,183490,185490,185750,192820,194480,196170,200130,204270,204320,205470,206640,206650,207940,213420,214150,214320,214370,214450,214980,215000,215200,216080,217270,218410,220630,222080,222800,226950,230360,232140,235980,237690,240810,241560,243070,247540,251270,252670,253450,256840,263720,263750,267250,267980,268600,271560,272210,272290,273130,278280,278530,282330,285130,287410,290510,290650,292150,293490,293780,294090,294870,298000,298020,298050,298380,299030,299660,299900,307950,314130,316140,319400,319660,321550,323990,326030,330590,330860,336260,336370,347860,348150,348210,352820,357780,363280,950130
0,2021-11-01,29889.657078,37788.947299,61587.095578,147803.768880,103845.308066,16875.345887,47152.899014,85094.421061,99148.736520,725721.243542,50555.726542,248348.687793,33855.491438,54439.590736,16763.206868,2405.908078,28396.932081,5304.012955,336471.904023,51888.744001,13258.674533,32377.962076,9794.846346,7671.029963,30769.611923,142164.772353,46380.018871,85025.857741,46228.695364,251719.658228,288527.937752,83227.805793,105378.435112,34064.574384,28992.870467,33934.452441,156979.864601,210124.133035,97844.607999,98809.412408,302795.121238,65305.587288,33702.768963,70107.938206,63818.287159,13267.564213,65409.234518,272254.448613,41902.065036,735235.249091,190665.195459,9193.381633,8910.205407,32891.550562,524605.246067,20148.505833,4753.845892,88672.536632,58309.866484,157351.868913,105901.441482,103613.916739,45039.243164,31089.379789,137166.820721,60288.961378,540875.198955,68524.265666,45734.493080,104869.431468,27785.566079,205726.672182,239357.555729,29383.582609,87053.678675,170312.818294,269569.120239,45552.763357,87117.497892,84371.570095,5387.657752,80408.065960,334660.869844,6201.144581,23587.514469,49016.908438,13290.264844,312265.007901,48175.769354,155788.732438,14789.446216,58170.077851,104477.981912,81171.414225,6756.495753,107397.146841,11455.178547,87748.689196,12418.208428,33218.295605,4561.076568,25271.176049,120891.352143,42788.761768,6309.820088,35169.8

In [13]:
model_sub.to_csv(path + 'lgbm.csv', index = False)